## Example submission

Image Matching Challenge 2025: https://www.kaggle.com/competitions/image-matching-challenge-2025

This notebook creates a simple submission using ALIKED and LightGlue, plus DINO for shortlisting, on GPU. Adapted from [last year](https://www.kaggle.com/code/oldufo/imc-2024-submission-example).

Remember to select an accelerator on the sidebar to the right, and to disable internet access when submitting a notebook to the competition.

In [1]:
# IMPORTANT 
#Install dependencies and copy model weights to run the notebook without internet access when submitting to the competition.

!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/aliked-n16.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth

Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia-0.7.2-py2.py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_moons-0.2.9-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_rs-0.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/pycolmap-0.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/rerun_sdk-0.15.0a2-cp38-abi3-manylinux_2_31_x86_64.whl
kornia is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
kornia-moons is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
kornia-rs is already installed 

In [2]:
import sys
import os
from tqdm import tqdm
from time import time, sleep
import gc
import numpy as np
import h5py
import dataclasses
import pandas as pd
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from PIL import Image

import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF

import torch
from lightglue import match_pair
from lightglue import ALIKED, LightGlue
from lightglue.utils import load_image, rbd
from transformers import AutoImageProcessor, AutoModel

from lightglue import DISK
from kornia.feature import LightGlueMatcher as KF_LightGlueMatcher
from scipy.spatial import cKDTree # For efficient nearest neighbor search to remove duplicate keypoints

# IMPORTANT Utilities: importing data into colmap and competition metric
import pycolmap
sys.path.append('/kaggle/input/imc25-utils')
from database import *
from h5_to_db import *
import metric

/usr/local/lib/python3.10/dist-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.10/dist-packages/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [3]:

print("PyTorch version:", torch.__version__)
import sys
print("Python version:", sys.version)

print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


PyTorch version: 2.5.1+cu121
Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
CUDA available: True
CUDA version: 12.1
Device count: 1
Current device: 0
Device name: Tesla P100-PCIE-16GB


In [4]:
# Do not forget to select an accelerator on the sidebar to the right.
device = K.utils.get_cuda_device_if_available(0)
print(f'{device=}')

device=device(type='cuda', index=0)


In [5]:
import torch
import torch.nn.functional as F
import numpy as np
import os
import h5py
from tqdm import tqdm
from PIL import Image
from transformers import AutoImageProcessor, AutoModel
import sys

# Assume these are available from your environment or previous code
# from .utils import load_torch_image # Assuming load_torch_image is defined elsewhere
# from kornia.feature import ALIKED # Already in your detect_aliked
# from kornia.feature import LightGlueMatcher as KF_LightGlueMatcher # Already in your match_with_lightglue
# from kornia.geometry import laf_from_center_scale_ori # Already in your match_with_lightglue
# from colmap_database import COLMAPDatabase, add_keypoints, add_matches # Already in your colmap_import

# --- Helper function for image loading (if not already defined) ---
def load_torch_image(fname, device=torch.device('cpu')):
    img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img

def get_dino_patch_features_for_keypoints(img_path, keypoints_xy, dino_processor, dino_model, patch_size=16, device=torch.device('cpu')):
    """
    Extracts DINO patch features corresponding to given ALIKED keypoint locations.
    It correctly infers the DINO patch grid dimensions from the processed input.

    Args:
        img_path (str): Path to the image file.
        keypoints_xy (torch.Tensor): Nx2 tensor of (x, y) keypoint coordinates in image pixel space.
                                     These keypoints are assumed to be in the original image's coordinate system.
        dino_processor: HuggingFace AutoImageProcessor for DINO.
        dino_model: HuggingFace AutoModel for DINO.
        patch_size (int): The patch size used by the DINO model (e.g., 14 or 16).
        device (torch.device): Device to run the models on.

    Returns:
        torch.Tensor: NxD_dino tensor of DINO patch features for each keypoint.
                      Returns None if no keypoints or image loading fails.
    """
    if len(keypoints_xy) == 0:
        dino_feature_dim = dino_model.config.hidden_size # Get actual DINO hidden size
        return torch.empty((0, dino_feature_dim), device=device)

    # 1. Load the original image (ALIKED processed this size)
    original_img = load_torch_image(img_path, device=device)
    original_h, original_w = original_img.shape[-2], original_img.shape[-1]


    # 2. Process the image with DINO's processor
    #    This step performs resizing, padding, etc., as needed by the DINO model
    with torch.inference_mode():
        # dino_processor returns a BatchFeature object which includes pixel_values
        # and potentially other information like `pixel_mask`
        inputs = dino_processor(images=original_img, return_tensors="pt", do_rescale=False).to(device)
        outputs = dino_model(**inputs)

        # Get the actual dimensions of the image as processed by the DINO model
        # This is the crucial part: the actual H and W that produced `patch_tokens`
        # We can infer this from the `pixel_values` shape
        processed_h = inputs['pixel_values'].shape[-2]
        processed_w = inputs['pixel_values'].shape[-1]

        # Extract patch tokens (excluding the CLS token)
        patch_tokens = outputs.last_hidden_state[:, 1:].squeeze(0) # Shape: (num_patches, hidden_size)

        # Calculate the actual grid dimensions based on the *processed* image size
        # and the model's patch size.
        # This should perfectly match the number of patch_tokens if the model is well-behaved.
        num_patches_h = processed_h // patch_size
        num_patches_w = processed_w // patch_size

        # Safety check: ensure calculated grid matches actual token count
        expected_token_count = num_patches_h * num_patches_w
        if patch_tokens.shape[0] != expected_token_count:
            # This indicates a deeper issue with how the model's output tokens
            # map to the spatial grid, or an unexpected patch size/model behavior.
            # Some models might have slightly different patch token arrangements.
            # DINOv2 typically aligns well.
            raise ValueError(
                f"DINO patch token count ({patch_tokens.shape[0]}) does not match "
                f"expected grid dimensions ({num_patches_h}x{num_patches_w} = {expected_token_count}) "
                f"for processed image size {processed_w}x{processed_h} with patch size {patch_size}. "
                f"Please verify DINO model and processor configuration."
            )

        # Reshape patch tokens into a 2D grid
        patch_features_grid = patch_tokens.reshape(num_patches_h, num_patches_w, -1)
        dino_feature_dim = patch_features_grid.shape[-1] # Actual feature dimension


    dino_features_for_kpts = torch.zeros((len(keypoints_xy), dino_feature_dim), device=device)

    # 3. Rescale ALIKED keypoints to the DINO *processed* image dimensions
    #    ALIKED keypoints are in original_w x original_h coordinates.
    #    DINO patches correspond to processed_w x processed_h coordinates.
    scale_x = processed_w / original_w
    scale_y = processed_h / original_h

    scaled_keypoints_xy = keypoints_xy.clone()
    scaled_keypoints_xy[:, 0] *= scale_x
    scaled_keypoints_xy[:, 1] *= scale_y

    # 4. Map scaled keypoints to DINO patch grid indices
    keypoint_cols = (scaled_keypoints_xy[:, 0] / patch_size).long()
    keypoint_rows = (scaled_keypoints_xy[:, 1] / patch_size).long()

    # Clip indices to ensure they are within bounds of the patch grid
    keypoint_rows = torch.clamp(keypoint_rows, 0, num_patches_h - 1)
    keypoint_cols = torch.clamp(keypoint_cols, 0, num_patches_w - 1)

    # Gather DINO features for each keypoint's corresponding patch
    dino_features_for_kpts = patch_features_grid[keypoint_rows, keypoint_cols]

    return dino_features_for_kpts


# --- MODIFIED: Detect ALIKED and Combine with DINO Patch Features ---
def detect_aliked_and_combine_with_dino(img_fnames,
                                        feature_dir='.featureout',
                                        num_features=4096,
                                        resize_to=1024,
                                        dino_processor=None,
                                        dino_model=None,
                                        dino_patch_size=16, # Typically 14 or 16 for DINO
                                        device=torch.device('cpu')):
    dtype = torch.float32 # ALIKED has issues with float16
    aliked_extractor = ALIKED(max_num_keypoints=num_features, detection_threshold=0.1).eval().to(device, dtype)
    aliked_extractor.preprocess_conf["resize"] = resize_to
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)

    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp, \
         h5py.File(f'{feature_dir}/descriptors_aliked.h5', mode='w') as f_desc_aliked, \
         h5py.File(f'{feature_dir}/descriptors_combined.h5', mode='w') as f_desc_combined: # New HDF5 for combined features
        for img_path in tqdm(img_fnames):
            img_fname = img_path.split('/')[-1]
            key = img_fname

            with torch.inference_mode():
                image0 = load_torch_image(img_path, device=device).to(dtype)
                feats0 = aliked_extractor.extract(image0)
                kpts = feats0['keypoints'].reshape(-1, 2).detach().cpu().numpy() # ALIKED keypoints (x,y)
                descs_aliked = feats0['descriptors'].reshape(len(kpts), -1).detach().cpu().numpy() # ALIKED descriptors

                # Get DINO patch features for these keypoints
                kpts_torch = torch.from_numpy(kpts).to(device)
                descs_dino_patch = get_dino_patch_features_for_keypoints(
                    img_path, kpts_torch, dino_processor, dino_model, dino_patch_size, device
                ).detach().cpu().numpy()

                # Concatenate ALIKED and DINO features
                if len(descs_aliked) > 0 and len(descs_dino_patch) > 0:
                    combined_descs = np.concatenate((descs_aliked, descs_dino_patch), axis=1)
                elif len(descs_aliked) > 0: # Only ALIKED if no DINO features (shouldn't happen often)
                    combined_descs = descs_aliked
                else: # No features found
                    combined_descs = np.array([]) # Empty array

                f_kp[key] = kpts
                f_desc_aliked[key] = descs_aliked # Keep ALIKED descriptors for debugging or other uses
                f_desc_combined[key] = combined_descs # Store the new combined descriptors
    print(f"Combined features saved to {feature_dir}/descriptors_combined.h5")
    return

In [6]:
from sklearn.cluster import MiniBatchKMeans # MiniBatchKMeans is faster for large datasets

# --- VLAD Aggregation Function ---
def vlad_encode(descriptors, centroids):
    """
    Performs VLAD encoding.

    Args:
        descriptors (np.ndarray): NxM array of local descriptors.
        centroids (np.ndarray): KxM array of K-Means cluster centroids.

    Returns:
        np.ndarray: 1x(K*M) VLAD descriptor.
    """
    if descriptors.shape[0] == 0:
        return np.zeros(centroids.shape[0] * centroids.shape[1], dtype=np.float32)

    num_descriptors, desc_dim = descriptors.shape
    num_centroids, _ = centroids.shape

    # Assign each descriptor to its nearest centroid
    # Using cdist for efficiency
    distances = np.sqrt(np.sum((descriptors[:, None, :] - centroids[None, :, :])**2, axis=2))
    # distances = cdist(descriptors, centroids, 'sqeuclidean') # Could use cdist for sqeuclidean
    cluster_assignments = np.argmin(distances, axis=1)

    # Initialize VLAD accumulator
    vlad_accumulator = np.zeros((num_centroids, desc_dim), dtype=np.float32)

    # Accumulate residuals
    for i in range(num_descriptors):
        cluster_idx = cluster_assignments[i]
        residual = descriptors[i] - centroids[cluster_idx]
        vlad_accumulator[cluster_idx] += residual

    # Flatten and L2 normalize
    vlad_descriptor = vlad_accumulator.flatten()
    vlad_descriptor = F.normalize(torch.from_numpy(vlad_descriptor).unsqueeze(0), dim=1, p=2).squeeze(0).numpy()

    return vlad_descriptor


# --- NEW: Get Global Descriptors using K-Means + VLAD ---
def get_global_desc_vlad(fnames, feature_dir='.featureout', num_clusters=64, device=torch.device('cpu')):
    """
    Generates global descriptors for images using K-Means + VLAD on combined ALIKED+DINO features.

    Args:
        fnames (list): List of image file paths.
        feature_dir (str): Directory where combined descriptors are stored.
        num_clusters (int): Number of clusters for K-Means (K in VLAD).
        device (torch.device): Not directly used for VLAD computation, but passed for consistency.

    Returns:
        torch.Tensor: Nx(K*M) tensor of global VLAD descriptors.
    """
    all_local_descs = []
    keys_order = [] # To maintain order of descriptors with respect to fnames

    # 1. Load all combined local descriptors
    with h5py.File(f'{feature_dir}/descriptors_combined.h5', mode='r') as f_desc_combined:
        for img_path in tqdm(fnames, desc="Loading combined local descriptors for K-Means"):
            key = img_path.split('/')[-1]
            if key in f_desc_combined:
                descs = f_desc_combined[key][...]
                if descs.shape[0] > 0:
                    all_local_descs.append(descs)
                    keys_order.append(key)

    if not all_local_descs:
        print("No combined local descriptors found. Cannot train K-Means or compute VLAD.")
        return torch.empty((0, num_clusters * 0), dtype=torch.float32) # Return empty tensor

    # Concatenate all descriptors for K-Means training
    all_local_descs_flat = np.concatenate(all_local_descs, axis=0)

    # 2. Train K-Means on a subset of descriptors if the dataset is too large
    # Or directly on all_local_descs_flat if memory permits
    print(f"Training K-Means with {num_clusters} clusters on {all_local_descs_flat.shape[0]} descriptors...")
    # Use MiniBatchKMeans for efficiency
    kmeans = MiniBatchKMeans(n_clusters=num_clusters, random_state=0, n_init='auto', batch_size=256).fit(all_local_descs_flat)
    centroids = kmeans.cluster_centers_
    print("K-Means training complete.")

    # 3. Compute VLAD descriptor for each image
    global_descs_vlad = []
    # Re-iterate through original fnames to match the output order
    with h5py.File(f'{feature_dir}/descriptors_combined.h5', mode='r') as f_desc_combined:
        for img_path in tqdm(fnames, desc="Computing VLAD descriptors"):
            key = img_path.split('/')[-1]
            if key in f_desc_combined:
                descs = f_desc_combined[key][...]
                vlad_desc = vlad_encode(descs, centroids)
                global_descs_vlad.append(torch.from_numpy(vlad_desc).unsqueeze(0))
            else:
                # Handle cases where an image might not have any combined descriptors
                # (e.g., no ALIKED keypoints detected). Append a zero vector of correct size.
                print(f"Warning: No combined descriptors for {key}. Appending zero VLAD descriptor.")
                # Determine descriptor dimension from centroids
                desc_dim_per_cluster = centroids.shape[1] if centroids.shape[1] > 0 else 0 # Should not be 0 normally
                zero_vlad = np.zeros(num_clusters * desc_dim_per_cluster, dtype=np.float32)
                global_descs_vlad.append(torch.from_numpy(zero_vlad).unsqueeze(0))


    if not global_descs_vlad:
        return torch.empty((0, num_clusters * centroids.shape[1] if centroids.shape[1] > 0 else 0), dtype=torch.float32)

    global_descs_vlad = torch.cat(global_descs_vlad, dim=0)
    return global_descs_vlad

In [7]:
# --- RE-DEFINED: get_image_pairs_shortlist to use the new VLAD global descriptor ---
def get_image_pairs_shortlist_vlad(fnames,
                                   sim_th=0.6, # should be strict
                                   min_pairs=30,
                                   exhaustive_if_less=20,
                                   feature_dir='.featureout', # Pass feature_dir
                                   num_clusters_vlad=64, # New parameter for VLAD
                                   device=torch.device('cpu')):
    num_imgs = len(fnames)
    if num_imgs <= exhaustive_if_less:
        return get_img_pairs_exhaustive(fnames) # You need to define get_img_pairs_exhaustive if not done.

    # Use the new VLAD-based global descriptor
    descs = get_global_desc_vlad(fnames, feature_dir=feature_dir, num_clusters=num_clusters_vlad, device=device)

    if descs.shape[0] == 0:
        print("No global descriptors generated. Returning empty matching list.")
        return []

    dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()

    # 只分析上三角（去掉对角线），避免重复
    triu_indices = np.triu_indices_from(dm, k=1)
    dm_flat = dm[triu_indices]
    
    # 打印统计信息
    print("Distance Matrix Statistics:")
    print(f"Min:  {dm_flat.min():.4f}")
    print(f"Max:  {dm_flat.max():.4f}")
    print(f"Mean: {dm_flat.mean():.4f}")
    print(f"Std:  {dm_flat.std():.4f}")
    print(f"20%:  {np.percentile(dm_flat, 20):.4f}")
    print(f"25%:  {np.percentile(dm_flat, 25):.4f}")
    print(f"50%:  {np.percentile(dm_flat, 50):.4f}")
    print(f"75%:  {np.percentile(dm_flat, 75):.4f}")
    threshold = dm_flat.mean() + np.sqrt(3) * dm_flat.std()

    # removing half
    mask = dm <= np.percentile(dm_flat, 50)
    total = 0
    matching_list = []
    ar = np.arange(num_imgs)
    already_there_set = set() # Use a set for faster lookup of already added pairs

    for st_idx in range(num_imgs - 1):
        mask_idx = mask[st_idx]
        to_match = ar[mask_idx]
        if len(to_match) < min_pairs:
            to_match = np.argsort(dm[st_idx])[:min_pairs]

        for idx in to_match:
            if st_idx == idx:
                continue
            if dm[st_idx, idx] < threshold: # Ensure distance is not effectively infinite
                pair = tuple(sorted((st_idx, idx.item())))
                if pair not in already_there_set:
                    matching_list.append(pair)
                    already_there_set.add(pair)
                    total += 1
    matching_list = sorted(list(matching_list)) # Sort the list of tuples
    return matching_list

In [8]:
# def load_torch_image(fname, device=torch.device('cpu')):
#     img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
#     return img


# # Must Use efficientnet global descriptor to get matching shortlists.
# def get_global_desc(fnames, device = torch.device('cpu')):
#     processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
#     model = AutoModel.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
#     model = model.eval()
#     model = model.to(device)
#     global_descs_dinov2 = []
#     for i, img_fname_full in tqdm(enumerate(fnames),total= len(fnames)):
#         key = os.path.splitext(os.path.basename(img_fname_full))[0]
#         timg = load_torch_image(img_fname_full)
#         with torch.inference_mode():
#             inputs = processor(images=timg, return_tensors="pt", do_rescale=False).to(device)
#             outputs = model(**inputs)
#             dino_mac = F.normalize(outputs.last_hidden_state[:,1:].max(dim=1)[0], dim=1, p=2)
#         global_descs_dinov2.append(dino_mac.detach().cpu())
#     global_descs_dinov2 = torch.cat(global_descs_dinov2, dim=0)
#     return global_descs_dinov2


def get_img_pairs_exhaustive(img_fnames):
    index_pairs = []
    for i in range(len(img_fnames)):
        for j in range(i+1, len(img_fnames)):
            index_pairs.append((i,j))
    return index_pairs


# def get_image_pairs_shortlist(fnames,
#                               sim_th = 0.6, # should be strict
#                               min_pairs = 30,
#                               exhaustive_if_less = 20,
#                               device=torch.device('cpu')):
#     num_imgs = len(fnames)
#     if num_imgs <= exhaustive_if_less:
#         return get_img_pairs_exhaustive(fnames)
#     descs = get_global_desc(fnames, device=device)
#     dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()

#     # 只分析上三角（去掉对角线），避免重复
#     triu_indices = np.triu_indices_from(dm, k=1)
#     dm_flat = dm[triu_indices]
    
#     # 打印统计信息
#     print("Distance Matrix Statistics:")
#     print(f"Min:  {dm_flat.min():.4f}")
#     print(f"Max:  {dm_flat.max():.4f}")
#     print(f"Mean: {dm_flat.mean():.4f}")
#     print(f"Std:  {dm_flat.std():.4f}")
#     print(f"20%:  {np.percentile(dm_flat, 20):.4f}")
#     print(f"25%:  {np.percentile(dm_flat, 25):.4f}")
#     print(f"50%:  {np.percentile(dm_flat, 50):.4f}")
#     print(f"75%:  {np.percentile(dm_flat, 75):.4f}")
#     threshold = dm_flat.mean() + np.sqrt(3) * dm_flat.std()
#     # removing half
#     # thrd = min(np.percentile(dm_flat, 60),sim_th)
#     # print(f"USED threshold: :  {thrd:.4f}")
#     mask = dm <= np.percentile(dm_flat, 50)
#     total = 0
#     matching_list = []
#     ar = np.arange(num_imgs)
#     already_there_set = []
#     for st_idx in range(num_imgs-1):
#         mask_idx = mask[st_idx]
#         to_match = ar[mask_idx]
#         if len(to_match) < min_pairs:
#             to_match = np.argsort(dm[st_idx])[:min_pairs]  
#         for idx in to_match:
#             if st_idx == idx:
#                 continue
#             if dm[st_idx, idx] < threshold:
#                 matching_list.append(tuple(sorted((st_idx, idx.item()))))
#                 total+=1
#     matching_list = sorted(list(set(matching_list)))
#     return matching_list

def detect_aliked(img_fnames,
                  feature_dir = '.featureout',
                  num_features = 4096,
                  resize_to = 1024,
                  device=torch.device('cpu')):
    dtype = torch.float32 # ALIKED has issues with float16
    extractor = ALIKED(max_num_keypoints=num_features, detection_threshold=0.1).eval().to(device, dtype)
    extractor.preprocess_conf["resize"] = resize_to
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)
    h5_kp_path = os.path.join(feature_dir, 'keypoints_aliked.h5')
    h5_desc_path = os.path.join(feature_dir, 'descriptors_aliked.h5')
    with h5py.File(h5_kp_path, mode='w') as f_kp, \
         h5py.File(h5_desc_path, mode='w') as f_desc:
        for img_path in tqdm(img_fnames):
            img_fname = img_path.split('/')[-1]
            key = img_fname
            with torch.inference_mode():
                image0 = load_torch_image(img_path, device=device).to(dtype)
                feats0 = extractor.extract(image0)  # auto-resize the image, disable with resize=None
                kpts = feats0['keypoints'].reshape(-1, 2).detach().cpu().numpy()
                descs = feats0['descriptors'].reshape(len(kpts), -1).detach().cpu().numpy()
                f_kp[key] = kpts
                f_desc[key] = descs
    return h5_kp_path, h5_desc_path

# def match_with_lightglue_aliked(img_fnames,
#                    index_pairs,
#                    feature_dir = '.featureout',
#                    device=torch.device('cpu'),
#                    min_matches=20,verbose=True):
#     lg_matcher = KF.LightGlueMatcher("aliked", {"width_confidence": -1,
#                                                 "depth_confidence": -1,
#                                                  "mp": True if 'cuda' in str(device) else False}).eval().to(device)
#     output_matches_path = os.path.join(feature_dir, 'matches_aliked.h5')
#     with h5py.File(f'{feature_dir}/keypoints_aliked.h5', mode='r') as f_kp, \
#         h5py.File(f'{feature_dir}/descriptors_aliked.h5', mode='r') as f_desc, \
#         h5py.File(output_matches_path, mode='w') as f_match:
#         for pair_idx in tqdm(index_pairs):
#             idx1, idx2 = pair_idx
#             fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
#             key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
#             kp1 = torch.from_numpy(f_kp[key1][...]).to(device)
#             kp2 = torch.from_numpy(f_kp[key2][...]).to(device)
#             desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
#             desc2 = torch.from_numpy(f_desc[key2][...]).to(device)
#             with torch.inference_mode():
#                 dists, idxs = lg_matcher(desc1,
#                                          desc2,
#                                          KF.laf_from_center_scale_ori(kp1[None]),
#                                          KF.laf_from_center_scale_ori(kp2[None]))
#             if len(idxs)  == 0:
#                 continue
#             n_matches = len(idxs)
#             if verbose:
#                 print (f'{key1}-{key2}: {n_matches} matches')
#             group  = f_match.require_group(key1)
#             if n_matches >= min_matches:
#                  group.create_dataset(key2, data=idxs.detach().cpu().numpy().reshape(-1, 2))
#     return output_matches_path

def match_with_lightglue_aliked(img_fnames,
                                index_pairs,
                                feature_kp_path, # Path to ALIKED keypoints H5
                                feature_desc_path, # Path to ALIKED descriptors H5
                                output_matches_h5='matches_aliked_lightglue.h5', # Specific output file
                                device=torch.device('cpu'),
                                min_matches=20,
                                verbose=True):
    """
    Performs feature matching using LightGlue with ALIKED descriptors.

    Args:
        img_fnames (list): List of original image file paths.
        index_pairs (list): List of (idx1, idx2) tuples representing image pairs to match.
        feature_kp_path (str): Path to the HDF5 file containing ALIKED keypoints.
        feature_desc_path (str): Path to the HDF5 file containing ALIKED descriptors.
        output_matches_h5 (str): Name of the HDF5 file to save the matches.
        device (torch.device): Device to run the matcher on.
        min_matches (int): Minimum number of matches required to save a pair.
        verbose (bool): If True, print progress and match counts.

    Returns:
        str: Path to the HDF5 file containing the saved matches.
    """
    # Initialize LightGlueMatcher for ALIKED features.
    # Note: KF needs to be imported or mocked from Kornia.feature
    lg_matcher = KF.LightGlueMatcher("aliked", {"width_confidence": -1,
                                                "depth_confidence": -1,
                                                 "mp": True if 'cuda' in str(device) else False}).eval().to(device)
    
    output_matches_path = os.path.join(os.path.dirname(feature_kp_path), output_matches_h5)
    
    with h5py.File(feature_kp_path, mode='r') as f_kp, \
         h5py.File(feature_desc_path, mode='r') as f_desc, \
         h5py.File(output_matches_path, mode='w') as f_match:
        
        for pair_idx in tqdm(index_pairs, desc="Matching ALIKED with LightGlue"):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = os.path.basename(fname1), os.path.basename(fname2)

            # Check if features exist for both images
            if key1 not in f_kp or key2 not in f_kp or key1 not in f_desc or key2 not in f_desc:
                if verbose:
                    print(f"Skipping {key1}-{key2}: features not found in HDF5.")
                continue

            # Load keypoints and descriptors
            # Convert to PyTorch tensors and move to device
            kp1 = torch.from_numpy(f_kp[key1][...]).to(device)
            kp2 = torch.from_numpy(f_kp[key2][...]).to(device)
            desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
            desc2 = torch.from_numpy(f_desc[key2][...]).to(device)

            # Skip if either image has no keypoints or descriptors
            if kp1.shape[0] == 0 or kp2.shape[0] == 0 or desc1.shape[0] == 0 or desc2.shape[0] == 0:
                if verbose:
                    print(f'{key1}-{key2}: Skipping due to no features detected.')
                continue

            with torch.inference_mode():
                # Convert keypoints to Local Affine Frames (LAFs) as required by LightGlueMatcher
                # Note: laf_from_center_scale_ori needs to be imported from kornia.geometry
                lafs1 = KF.laf_from_center_scale_ori(kp1[None]) # [None] adds a batch dimension
                lafs2 = KF.laf_from_center_scale_ori(kp2[None])

                # Perform matching
                dists, idxs = lg_matcher(desc1, desc2, lafs1, lafs2)
            
            # If no matches are found, continue to the next pair
            if len(idxs) == 0:
                continue
            
            n_matches = len(idxs)
            if verbose:
                print (f'{key1}-{key2}: {n_matches} matches')
            
            # Create a group for the first image if it doesn't exist
            group = f_match.require_group(key1)
            
            # Save matches if they meet the minimum count
            if n_matches >= min_matches:
                 group.create_dataset(key2, data=idxs.detach().cpu().numpy().reshape(-1, 2))
                 
    print(f"ALIKED + LightGlue matches saved to {output_matches_path}")
    return output_matches_path


def import_into_colmap(img_dir, feature_dir ='.featureout', database_path = 'colmap.db'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, '', 'simple-pinhole', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )
    db.commit()
    return

In [9]:
# --- NEW: Detect DISK features ---
def detect_disk(img_fnames, feature_dir = '.featureout', num_features = 4096, resize_to = 1024, device=torch.device('cpu')):
    """
    Detects DISK features using the LightGlue-provided DISK wrapper,
    which accepts max_num_keypoints in its constructor and uses .extract().

    Args:
        img_fnames (list): List of image file paths.
        feature_dir (str): Directory to save output HDF5 files.
        num_features (int): Maximum number of features to detect per image.
        resize_to (int): Image size to resize to for feature detection.
        device (torch.device): Device to run the model on.

    Returns:
        tuple: Paths to the keypoints H5 file and descriptors H5 file.
    """
    # Initialize DISK as it appears in the LightGlue library's usage pattern.
    # This version correctly uses `max_num_keypoints` in the constructor
    # and calls `.extract()` on the extractor.
    extractor = DISK(max_num_keypoints=num_features, # This is the argument LightGlue's wrapper expects
                     detection_threshold=0.001, # From notebook's CONFIG.params_disk_lightglue
                     resize=resize_to # Pass resize to the constructor
                    ).eval().to(device)
    
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)
        
    h5_kp_path = os.path.join(feature_dir, 'keypoints_disk.h5')
    h5_desc_path = os.path.join(feature_dir, 'descriptors_disk.h5')

    with h5py.File(h5_kp_path, mode='w') as f_kp, \
         h5py.File(h5_desc_path, mode='w') as f_desc:
        for img_path in tqdm(img_fnames, desc="Detecting DISK features"):
            key = os.path.basename(img_path)
            with torch.inference_mode():
                image0 = load_torch_image(img_path, device=device) # Load as Kornia expects (RGB, float, 0-1)
                
                # Call .extract() on the extractor, as seen in the notebook
                feats0 = extractor.extract(image0) 
                
                kpts = feats0['keypoints'].reshape(-1, 2).detach().cpu().numpy()
                descs = feats0['descriptors'].reshape(len(kpts), -1).detach().cpu().numpy()

                # Ensure empty arrays are correctly shaped if no features found
                if len(kpts) == 0:
                    kpts = np.array([], dtype=np.float32).reshape(0, 2)
                    # Use the actual descriptor dimension found, or default to 256
                    descs = np.array([], dtype=np.float32).reshape(0, descs.shape[-1] if descs.shape[-1] > 0 else 256)
                
                f_kp[key] = kpts
                f_desc[key] = descs
                
    print(f"DISK features saved to {h5_kp_path} and {h5_desc_path}")
    return h5_kp_path, h5_desc_path


# --- MODIFIED: Detect SIFT features using OpenCV ---
def detect_sift(img_fnames,
                feature_dir = '.featureout',
                num_features = 4096,
                resize_to = 1024, # SIFT in OpenCV usually works best on grayscale
                device=torch.device('cpu')): # Device parameter is not directly used by OpenCV, but kept for consistency
    
    # Initialize OpenCV SIFT detector
    # max_num_keypoints can be controlled via nfeatures
    sift_extractor = cv2.SIFT_create(nfeatures=num_features)
    
    if not os.path.isdir(feature_dir): os.makedirs(feature_dir)
    h5_kp_path = os.path.join(feature_dir, 'keypoints_sift.h5')
    h5_desc_path = os.path.join(feature_dir, 'descriptors_sift.h5')

    with h5py.File(h5_kp_path, mode='w') as f_kp, \
         h5py.File(h5_desc_path, mode='w') as f_desc:
        for img_path in tqdm(img_fnames, desc="Detecting SIFT features (OpenCV)"):
            key = os.path.basename(img_path)
            
            # Load image using PIL, convert to grayscale NumPy array for OpenCV
            pil_img = Image.open(img_path).convert('L') # Convert to grayscale
            np_img = np.array(pil_img) # H, W, uint8

            # Optional: Resize image before SIFT detection if resize_to is specified
            # OpenCV SIFT handles scaling internally, but if you want to limit input size:
            if resize_to is not None:
                original_h, original_w = np_img.shape
                # Calculate new dimensions preserving aspect ratio and fitting within resize_to
                scale = resize_to / max(original_h, original_w)
                new_w, new_h = int(original_w * scale), int(original_h * scale)
                np_img = cv2.resize(np_img, (new_w, new_h), interpolation=cv2.INTER_AREA)

            # Detect keypoints and compute descriptors
            # kp will be a list of cv2.KeyPoint objects
            # descs will be a NumPy array of shape (num_keypoints, 128)
            kp_list, descs = sift_extractor.detectAndCompute(np_img, None)

            # Convert cv2.KeyPoint objects to a NumPy array of (x, y) coordinates
            # KeyPoint.pt gives (x, y) tuple
            kpts = np.array([kp.pt for kp in kp_list], dtype=np.float32).reshape(-1, 2)
            
            # If no descriptors, set to empty array (important for HDF5)
            if descs is None:
                descs = np.array([], dtype=np.float32).reshape(0, 128) # SIFT descriptors are 128-dim

            # If resized, rescale keypoint coordinates back to original image size
            if resize_to is not None and len(kpts) > 0:
                scale_back_x = original_w / np_img.shape[1]
                scale_back_y = original_h / np_img.shape[0]
                kpts[:, 0] *= scale_back_x
                kpts[:, 1] *= scale_back_y


            f_kp[key] = kpts
            f_desc[key] = descs
    print(f"SIFT features (OpenCV) saved to {h5_kp_path} and {h5_desc_path}")
    return h5_kp_path, h5_desc_path


# # --- NEW: Detect SIFT features (using Kornia's SIFT, which wraps OpenCV/PyTorch backend) ---
# def detect_sift(img_fnames,
#                 feature_dir = '.featureout',
#                 num_features = 4096,
#                 resize_to = 1024,
#                 device=torch.device('cpu')):
#     # Kornia SIFT supports both torch and opencv backend. Using torch by default.
#     extractor = KorniaSIFT(num_features=num_features,
#                           upright=False, # Standard SIFT is not upright
#                           edge_threshold=10,
#                           pyr_levels=5,
#                           sigma=1.6).eval().to(device)
#     if not os.path.isdir(feature_dir):
#         os.makedirs(feature_dir)
#     # Use specific filenames for SIFT features
#     h5_kp_path = os.path.join(feature_dir, 'keypoints_sift.h5')
#     h5_desc_path = os.path.join(feature_dir, 'descriptors_sift.h5')

#     with h5py.File(h5_kp_path, mode='w') as f_kp, \
#          h5py.File(h5_desc_path, mode='w') as f_desc:
#         for img_path in tqdm(img_fnames, desc="Detecting SIFT features"):
#             img_fname = os.path.basename(img_path)
#             key = img_fname
#             with torch.inference_mode():
#                 image0 = load_torch_image(img_path, device=device).mean(dim=0, keepdim=True) # SIFT usually works on grayscale, Kornia's SIFT expects 1-channel or 3-channel
#                 # Make sure image is in float32 and [0,1] or [0,255] range as expected by Kornia SIFT.
#                 # load_torch_image already gives float32 [0,1].
#                 kpts_laf, descs = extractor(image0) # Kornia SIFT returns LAF and descriptors

#                 # Convert LAF to simple (x,y) keypoints
#                 # LAF is (1, N, 2, 3), keypoints are center (x,y) at [:, :, :, 2]
#                 kpts = kpts_laf[0, :, :2, 2].detach().cpu().numpy() # [batch_idx, keypoint_idx, center_x_y, 3rd_dim_for_laf_matrix]
#                 descs = descs[0].detach().cpu().numpy() # [batch_idx, descriptor_data]

#                 f_kp[key] = kpts
#                 f_desc[key] = descs
#     print(f"SIFT features saved to {h5_kp_path} and {h5_desc_path}")
#     return h5_kp_path, h5_desc_path

In [10]:
# --- NEW: Match DISK with LightGlue ---
def match_with_lightglue_disk(img_fnames,
                              index_pairs,
                              feature_kp_path, # Path to DISK keypoints H5
                              feature_desc_path, # Path to DISK descriptors H5
                              output_matches_h5='matches_disk.h5', # Specific output file
                              device=torch.device('cpu'),
                              min_matches=20,
                              verbose=True):
    lg_matcher = KF_LightGlueMatcher("disk", {"width_confidence": -1, # Using 'disk' type for LightGlue
                                                "depth_confidence": -1,
                                                "mp": True if 'cuda' in str(device) else False}).eval().to(device)
    output_matches_path = os.path.join(os.path.dirname(feature_kp_path), output_matches_h5)
    with h5py.File(feature_kp_path, mode='r') as f_kp, \
         h5py.File(feature_desc_path, mode='r') as f_desc, \
         h5py.File(output_matches_path, mode='w') as f_match:
        for pair_idx in tqdm(index_pairs, desc="Matching DISK with LightGlue"):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = os.path.basename(fname1), os.path.basename(fname2)

            if key1 not in f_kp or key2 not in f_kp or key1 not in f_desc or key2 not in f_desc:
                if verbose: print(f"Skipping {key1}-{key2}: features not found in HDF5.")
                continue

            kp1 = torch.from_numpy(f_kp[key1][...]).to(device)
            kp2 = torch.from_numpy(f_kp[key2][...]).to(device)
            desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
            desc2 = torch.from_numpy(f_desc[key2][...]).to(device)

            if kp1.shape[0] == 0 or kp2.shape[0] == 0 or desc1.shape[0] == 0 or desc2.shape[0] == 0:
                if verbose: print(f'{key1}-{key2}: Skipping due to no features detected.')
                continue

            with torch.inference_mode():
                dists, idxs = lg_matcher(desc1,
                                         desc2,
                                         KF.laf_from_center_scale_ori(kp1[None]), # DISK returns (x,y), so same LAF conversion
                                         KF.laf_from_center_scale_ori(kp2[None]))
            if len(idxs)  == 0: continue
            n_matches = len(idxs)
            if verbose: print (f'{key1}-{key2}: {n_matches} matches')
            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                 group.create_dataset(key2, data=idxs.detach().cpu().numpy().reshape(-1, 2))
    print(f"DISK + LightGlue matches saved to {output_matches_path}")
    return output_matches_path

# --- NEW: Match SIFT with FLANN + Ratio Test ---
def match_sift_flann(img_fnames,
                     index_pairs,
                     feature_kp_path,        # Path to SIFT keypoints H5
                     feature_desc_path,      # Path to SIFT descriptors H5
                     output_matches_h5='matches_sift.h5',
                     ratio_test_threshold=0.8,
                     device=torch.device('cpu'),  # unused but preserved
                     min_matches=20,
                     verbose=True):

    # FLANN index params for SIFT (uses KD-Tree)
    index_params = dict(algorithm=1, trees=5)  # 1 = FLANN_INDEX_KDTREE
    search_params = dict(checks=50)           # higher = more accurate but slower
    matcher = cv2.FlannBasedMatcher(index_params, search_params)

    output_matches_path = os.path.join(os.path.dirname(feature_kp_path), output_matches_h5)

    with h5py.File(feature_kp_path, mode='r') as f_kp, \
         h5py.File(feature_desc_path, mode='r') as f_desc, \
         h5py.File(output_matches_path, mode='w') as f_match:

        for pair_idx in tqdm(index_pairs, desc="Matching SIFT with FLANN"):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = os.path.basename(fname1), os.path.basename(fname2)

            if key1 not in f_kp or key2 not in f_kp or key1 not in f_desc or key2 not in f_desc:
                if verbose:
                    print(f"Skipping {key1}-{key2}: features not found in HDF5.")
                continue

            desc1 = f_desc[key1][...].astype(np.float32)
            desc2 = f_desc[key2][...].astype(np.float32)

            if desc1.shape[0] == 0 or desc2.shape[0] == 0:
                if verbose:
                    print(f'{key1}-{key2}: Skipping due to no features detected.')
                continue

            # FLANN requires descriptors to be float32 and non-empty
            try:
                matches = matcher.knnMatch(desc1, desc2, k=2)
            except cv2.error as e:
                if verbose:
                    print(f"{key1}-{key2}: FLANN matching failed with error: {e}")
                continue

            good_matches = [m for m, n in matches if m.distance < ratio_test_threshold * n.distance]

            n_matches = len(good_matches)
            if n_matches == 0:
                continue

            if verbose:
                print(f'{key1}-{key2}: {n_matches} SIFT FLANN matches')

            idxs = np.array([[m.queryIdx, m.trainIdx] for m in good_matches], dtype=np.int32)

            group = f_match.require_group(key1)
            if n_matches >= min_matches:
                group.create_dataset(key2, data=idxs)

    print(f"SIFT + FLANN matches saved to {output_matches_path}")
    return output_matches_path


In [11]:
# Collect vital info from the dataset

@dataclasses.dataclass
class Prediction:
    image_id: str | None  # A unique identifier for the row -- unused otherwise. Used only on the hidden test set.
    dataset: str
    filename: str
    cluster_index: int | None = None
    rotation: np.ndarray | None = None
    translation: np.ndarray | None = None

# Set is_train=True to run the notebook on the training data.
# Set is_train=False if submitting an entry to the competition (test data is hidden, and different from what you see on the "test" folder).
is_train = True
data_dir = '/kaggle/input/image-matching-challenge-2025'
workdir = '/kaggle/working/result/'
os.makedirs(workdir, exist_ok=True)

if is_train:
    sample_submission_csv = os.path.join(data_dir, 'train_labels.csv')
else:
    sample_submission_csv = os.path.join(data_dir, 'sample_submission.csv')

samples = {}
competition_data = pd.read_csv(sample_submission_csv)
for _, row in competition_data.iterrows():
    # Note: For the test data, the "scene" column has no meaning, and the rotation_matrix and translation_vector columns are random.
    if row.dataset not in samples:
        samples[row.dataset] = []
    samples[row.dataset].append(
        Prediction(
            image_id=None if is_train else row.image_id,
            dataset=row.dataset,
            filename=row.image
        )
    )

for dataset in samples:
    print(f'Dataset "{dataset}" -> num_images={len(samples[dataset])}')

Dataset "imc2023_haiper" -> num_images=54
Dataset "imc2023_heritage" -> num_images=209
Dataset "imc2023_theather_imc2024_church" -> num_images=76
Dataset "imc2024_dioscuri_baalshamin" -> num_images=138
Dataset "imc2024_lizard_pond" -> num_images=214
Dataset "pt_brandenburg_british_buckingham" -> num_images=225
Dataset "pt_piazzasanmarco_grandplace" -> num_images=168
Dataset "pt_sacrecoeur_trevi_tajmahal" -> num_images=225
Dataset "pt_stpeters_stpauls" -> num_images=200
Dataset "amy_gardens" -> num_images=200
Dataset "fbk_vineyard" -> num_images=163
Dataset "ETs" -> num_images=22
Dataset "stairs" -> num_images=51


In [12]:
# --- Core Ensemble and Remapping Function ---
def ensemble_and_remap_matches(img_fnames,
                               match_h5_paths_by_detector, # Dict: {'method_name': 'path/to/matches.h5'}
                               kp_h5_paths_by_detector, # Dict: {'detector_name': 'path/to/keypoints.h5'}
                               output_unified_kp_h5='keypoints_unified.h5',
                               output_remapped_matches_h5='matches_final_ensemble.h5',
                               kpt_merge_threshold=2.0, # Pixels
                               min_matches_per_pair=1):
    print("\n--- Ensembling and Remapping Matches ---")

    # Determine common output directory
    output_dir = os.path.dirname(list(match_h5_paths_by_detector.values())[0])
    unified_kp_path = os.path.join(output_dir, output_unified_kp_h5)
    remapped_matches_path = os.path.join(output_dir, output_remapped_matches_h5)

    image_keys = [os.path.basename(f) for f in img_fnames]

    unified_keypoints_per_image = {}
    old_to_new_index_maps_per_image = {}

    # --- Phase 1: Create Unified Keypoints and Build Index Maps ---
    for img_key in tqdm(image_keys, desc="Phase 1: Unifying keypoints per image"):
        all_kpts_for_img = []
        
        # Collect keypoints from all detectors for the current image
        detector_kpts_list = [] # List of (detector_name, kpts_array)
        for detector_name, kp_path in kp_h5_paths_by_detector.items():
            with h5py.File(kp_path, 'r') as f_kp:
                if img_key in f_kp and f_kp[img_key].shape[0] > 0:
                    kpts_detector = f_kp[img_key][...]
                    all_kpts_for_img.append(kpts_detector)
                    detector_kpts_list.append((detector_name, kpts_detector))
        
        if not all_kpts_for_img:
            unified_keypoints_per_image[img_key] = np.array([], dtype=np.float32).reshape(0,2) # Ensure empty (0,2) array
            old_to_new_index_maps_per_image[img_key] = {}
            continue

        concatenated_kpts = np.concatenate(all_kpts_for_img, axis=0)
        
        # Build KDTree for efficient merging
        tree = cKDTree(concatenated_kpts)
        close_pairs = tree.query_pairs(kpt_merge_threshold)

        # Disjoint Set Union (DSU) for clustering close keypoints
        parent = list(range(len(concatenated_kpts)))
        def find(i):
            if parent[i] == i: return i
            parent[i] = find(parent[i])
            return parent[i]
        def union(i, j):
            root_i, root_j = find(i), find(j)
            if root_i != root_j: parent[root_j] = root_i; return True
            return False
        for i, j in close_pairs: union(i, j)

        unified_kpts_raw_groups = {} # root_idx -> list of kpts
        for i in range(len(concatenated_kpts)):
            root_idx = find(i)
            if root_idx not in unified_kpts_raw_groups:
                unified_kpts_raw_groups[root_idx] = []
            unified_kpts_raw_groups[root_idx].append(concatenated_kpts[i])

        final_unified_kpts_list = []
        unified_idx_map = {} # root_idx -> new_unified_idx
        for root_idx, kpts_group in unified_kpts_raw_groups.items():
            unified_pos = np.mean(kpts_group, axis=0) # Average position
            new_unified_idx = len(final_unified_kpts_list)
            final_unified_kpts_list.append(unified_pos)
            unified_idx_map[root_idx] = new_unified_idx
        
        final_unified_kpts = np.array(final_unified_kpts_list, dtype=np.float32)
        unified_keypoints_per_image[img_key] = final_unified_kpts

        # Build old_idx -> new_idx mapping for each original detector's keypoints
        detector_to_new_idx_map = {}
        current_kpt_offset = 0 # Offset into concatenated_kpts
        for detector_name, original_kpts_array in detector_kpts_list:
            old_to_new_map_for_detector = np.full(original_kpts_array.shape[0], -1, dtype=np.int32)
            for original_local_idx in range(original_kpts_array.shape[0]):
                concatenated_idx = current_kpt_offset + original_local_idx
                root_idx = find(concatenated_idx)
                old_to_new_map_for_detector[original_local_idx] = unified_idx_map[root_idx]
            detector_to_new_idx_map[detector_name] = old_to_new_map_for_detector
            current_kpt_offset += original_kpts_array.shape[0]
        old_to_new_index_maps_per_image[img_key] = detector_to_new_idx_map

    # Save unified keypoints
    with h5py.File(unified_kp_path, 'w') as f_unified_kp:
        for img_key, unified_kpts_array in unified_keypoints_per_image.items():
            f_unified_kp.create_dataset(img_key, data=unified_kpts_array)
    print(f"Unified keypoints saved to {unified_kp_path}")

    # --- Phase 2: Remap and Ensemble Matches ---
    final_ensembled_remapped_matches_dict = {} # Key: (img1_key, img2_key) -> set of (new_idx1, new_idx2)

    # Need image_keys to iterate over pairs in a consistent order
    for i in tqdm(range(len(image_keys)), desc="Phase 2: Remapping and Ensembling Matches"):
        img1_key = image_keys[i]
        for j in range(i + 1, len(image_keys)):
            img2_key = image_keys[j]

            current_pair_remapped_matches_set = set() # Use a set to handle union and avoid duplicates

            # Iterate through each individual matcher's results
            for method_name, match_path in match_h5_paths_by_detector.items():
                detector_name = method_name.split('_')[0] # e.g., 'aliked', 'disk', 'sift', 'dedode'

                with h5py.File(match_path, 'r') as f_current_matches:
                    if img1_key in f_current_matches and img2_key in f_current_matches[img1_key]:
                        original_matches = f_current_matches[img1_key][img2_key][...] # (N, 2)
                        
                        # Get the remapping maps for this image pair and this detector
                        map1 = old_to_new_index_maps_per_image[img1_key].get(detector_name)
                        map2 = old_to_new_index_maps_per_image[img2_key].get(detector_name)

                        # Check if mappings exist and are not empty
                        if map1 is not None and map2 is not None and map1.shape[0] > 0 and map2.shape[0] > 0:
                            for orig_idx1, orig_idx2 in original_matches:
                                # Ensure original indices are within bounds of the map
                                if orig_idx1 < map1.shape[0] and orig_idx2 < map2.shape[0]:
                                    new_idx1 = map1[orig_idx1]
                                    new_idx2 = map2[orig_idx2]
                                    if new_idx1 != -1 and new_idx2 != -1: # Valid remapped indices
                                        current_pair_remapped_matches_set.add(tuple(sorted((new_idx1, new_idx2)))) # Store sorted for canonical form
                                else:
                                    # This indicates a potential issue with original match indices or mappings
                                    # print(f"Warning: Match index out of bounds for {img1_key}-{img2_key} from {method_name}")
                                    pass # Skip problematic match


            if len(current_pair_remapped_matches_set) >= min_matches_per_pair:
                final_ensembled_remapped_matches_dict[(img1_key, img2_key)] = list(current_pair_remapped_matches_set)

    # Save the final ensembled and remapped matches
    with h5py.File(remapped_matches_path, 'w') as f_remapped_matches:
        for (img1_key, img2_key), matches_list in tqdm(final_ensembled_remapped_matches_dict.items(), desc="Saving remapped ensembled matches"):
            if matches_list:
                matches_array = np.array(matches_list, dtype=np.int32)
                group = f_remapped_matches.require_group(img1_key)
                group.create_dataset(img2_key, data=matches_array)
            else: # Create empty dataset if no matches, for consistency
                group = f_remapped_matches.require_group(img1_key)
                group.create_dataset(img2_key, data=np.array([]), dtype=np.int32, shape=(0,2))

    print(f"Ensembled and remapped matches saved to {remapped_matches_path}")
    return unified_kp_path, remapped_matches_path

In [29]:
import cv2
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches


def draw_keypoints_and_matches(images_input, unified_kp_path, remapped_matches_path, feature_dir='visualization_output'):
    output_dir = os.path.join(feature_dir, 'visualization_output')
    os.makedirs(output_dir, exist_ok=True)

    # Load images and determine image_keys for HDF5 lookup
    if isinstance(images_input[0], str):
        loaded_images = [cv2.imread(img_path) for img_path in images_input]
        image_keys = [os.path.basename(img_path) for img_path in images_input]
    else:
        loaded_images = images_input
        # If images_input are already arrays, you need to provide the corresponding keys
        # This part is crucial: image_keys MUST align with the HDF5 keys
        image_keys = image_keys_in_h5 # Use the predefined list for the dummy case

    # Load unified keypoints
    keypoints_data = {}
    with h5py.File(unified_kp_path, 'r') as f_kp:
        for img_name_raw in f_kp.keys():
            img_name = img_name_raw.decode('utf-8') if isinstance(img_name_raw, bytes) else img_name_raw
            keypoints_data[img_name] = f_kp[img_name_raw][()] # Access with raw key if bytes

    # Load remapped matches - CORRECTED LOGIC
    # Store (img1_key, img2_key) directly with matches for robust iteration
    matches_data_pairs = [] # Will store (img1_key, img2_key, matches_array)
    with h5py.File(remapped_matches_path, 'r') as f_matches:
        print("\n--- Loading remapped matches from HDF5 ---")
        for img1_group_key_candidate in tqdm(f_matches.keys(), desc="Loading matches"):
            img1_key = img1_group_key_candidate.decode('utf-8') if isinstance(img1_group_key_candidate, bytes) else img1_group_key_candidate

            img1_group = f_matches[img1_group_key_candidate] # Access with raw key

            if isinstance(img1_group, h5py.Group):
                for img2_dataset_key_candidate in img1_group.keys():
                    img2_key = img2_dataset_key_candidate.decode('utf-8') if isinstance(img2_dataset_key_candidate, bytes) else img2_dataset_key_candidate

                    try:
                        matches_array = img1_group[img2_dataset_key_candidate][()]
                        matches_data_pairs.append((img1_key, img2_key, matches_array))
                    except Exception as e:
                        print(f"Error loading matches for pair ({img1_key}, {img2_key}): {e}")
            else:
                print(f"Warning: Expected '{img1_key}' to be a group, but found {type(img1_group)}. Skipping its contents.")


    # --- Drawing Keypoints ---
    print("\n--- Drawing Keypoints ---")
    for i, img_key in enumerate(image_keys):
        if img_key in keypoints_data:
            img = loaded_images[i].copy()
            kpts = keypoints_data[img_key]

            for kp in kpts:
                x, y = int(kp[0]), int(kp[1])
                cv2.circle(img, (x, y), 3, (0, 255, 0), -1) # Green circle for keypoint

            output_kp_path = os.path.join(output_dir, f"keypoints_{img_key}")
            if len(img.shape) == 2:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            cv2.imwrite(output_kp_path, img)
            print(f"Keypoints drawn on {img_key}, saved to {output_kp_path}")
        else:
            print(f"No keypoints found for {img_key} in unified keypoints file.")

    # --- Drawing Matches ---
    print("\n--- Drawing Matches ---")
    # Iterate through the (img1_key, img2_key, matches) tuples directly
    for img_name1, img_name2, matches in matches_data_pairs:
        # We no longer need to split img_pair_key, as we have img_name1 and img_name2 directly

        # Find the actual image objects and their keypoints using image_keys list
        try:
            img1_idx = image_keys.index(img_name1)
            img2_idx = image_keys.index(img_name2)
        except ValueError:
            print(f"Skipping matches for {img_name1}-{img_name2}: One or both image names not found in the provided 'images' list/keys.")
            continue

        img1 = loaded_images[img1_idx].copy()
        img2 = loaded_images[img2_idx].copy()

        kpts1 = keypoints_data.get(img_name1)
        kpts2 = keypoints_data.get(img_name2)

        if kpts1 is None or kpts2 is None:
            print(f"Skipping matches for {img_name1}-{img_name2}: keypoints not found for one or both images in unified keypoints.")
            continue
        if len(matches) == 0:
            print(f"No matches to draw for {img_name1}-{img_name2}.")
            continue

        # Ensure images are 3 channels for drawing lines
        if len(img1.shape) == 2:
            img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
        if len(img2.shape) == 2:
            img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

        # Create a concatenated image for drawing matches
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        max_h = max(h1, h2)
        matched_img = np.zeros((max_h, w1 + w2, 3), dtype=np.uint8)
        matched_img[0:h1, 0:w1] = img1
        matched_img[0:h2, w1:w1+w2] = img2

        num_matches_to_draw = min(len(matches), 200) # Draw up to 200 matches to avoid clutter, adjust as needed

        for i in range(num_matches_to_draw):
            match = matches[i]
            kp1_idx, kp2_idx = int(match[0]), int(match[1])

            # Bounds check for keypoint indices
            if kp1_idx >= len(kpts1) or kp2_idx >= len(kpts2):
                # print(f"Warning: Match index out of bounds for {img_name1}-{img_name2}. Skipping match {kp1_idx}-{kp2_idx}.")
                continue

            pt1 = tuple(map(int, kpts1[kp1_idx][:2]))
            pt2 = tuple(map(int, kpts2[kp2_idx][:2]))

            # Draw circles on the concatenated image
            cv2.circle(matched_img, pt1, 5, (0, 0, 255), 2) # Red circle on img1 side
            cv2.circle(matched_img, (pt2[0] + w1, pt2[1]), 5, (255, 0, 0), 2) # Blue circle on img2 side

            # Draw a line connecting the matched keypoints
            color = tuple(np.random.randint(0, 255, 3).tolist())
            cv2.line(matched_img, pt1, (pt2[0] + w1, pt2[1]), color, 1)

        output_match_path = os.path.join(output_dir, f"matches_{img_name1}_{img_name2}.png")
        cv2.imwrite(output_match_path, matched_img)
        print(f"Matches drawn between {img_name1} and {img_name2}, saved to {output_match_path}")


# Example call (replace with your actual 'images' list)
# If your 'images' are file paths:
# images_file_paths = ['path/to/your/image1.jpg', 'path/to/your/image2.jpg', ...]
# draw_keypoints_and_matches(images_file_paths, unified_kp_path, remapped_matches_path)

# If your 'images' are loaded numpy arrays (as in the dummy example above):
# draw_keypoints_and_matches(images, unified_kp_path, remapped_matches_path)

In [30]:
gc.collect()

max_images = None  # Used For debugging only. Set to None to disable.
datasets_to_process = None  # Not the best convention, but None means all datasets.

if is_train:
    # max_images = 5

    # Note: When running on the training dataset, the notebook will hit the time limit and die. Use this filter to run on a few specific datasets.
    datasets_to_process = [
    	# New data.
    	# 'amy_gardens',
    	'ETs',
    	# 'fbk_vineyard',
    	# 'stairs',
    	# Data from IMC 2023 and 2024.
    	# 'imc2024_dioscuri_baalshamin',
    	# 'imc2023_theather_imc2024_church',
    	# 'imc2023_heritage',
    	# 'imc2023_haiper',
    	# 'imc2024_lizard_pond',
    	# Crowdsourced PhotoTourism data.
    	# 'pt_stpeters_stpauls',
    	# 'pt_brandenburg_british_buckingham',
    	# 'pt_piazzasanmarco_grandplace',
    	# 'pt_sacrecoeur_trevi_tajmahal',
    ]

timings = {
    "global feature extraction":[],
    "shortlisting":[],
    "feature_detection": [],
    "feature_matching":[],
    "RANSAC": [],
    "Reconstruction": [],
}
mapping_result_strs = []

# Load DINOv2 model (for feature extraction, not global descriptor here)
print("Loading DINOv2 model for patch feature extraction...")
dino_processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
dino_model = AutoModel.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
dino_model = dino_model.eval().to(device)
print("DINOv2 model loaded.")

print (f"Extracting on device {device}")
for dataset, predictions in samples.items():
    if datasets_to_process and dataset not in datasets_to_process:
        print(f'Skipping "{dataset}"')
        continue
    
    images_dir = os.path.join(data_dir, 'train' if is_train else 'test', dataset)
    images = [os.path.join(images_dir, p.filename) for p in predictions]
    if max_images is not None:
        images = images[:max_images]

    print(f'\nProcessing dataset "{dataset}": {len(images)} images')

    filename_to_index = {p.filename: idx for idx, p in enumerate(predictions)}

    feature_dir = os.path.join(workdir, 'featureout', dataset)
    os.makedirs(feature_dir, exist_ok=True)

    # Wrap algos in try-except blocks so we can populate a submission even if one scene crashes.
    try:

    # --- Pipeline Execution ---

        # 1. Detect ALIKED features and combine with DINO patch features
        t = time()
        print("\n--- Step 1: Detecting ALIKED and Combining with DINO Patch Features ---")
        detect_aliked_and_combine_with_dino(
            img_fnames=images,
            feature_dir=feature_dir,
            num_features=4096,
            resize_to=1024,
            dino_processor=dino_processor,
            dino_model=dino_model,
            dino_patch_size=14, # Adjust based on your DINO model's patch size (e.g., 14 for DINOv2 base)
            device=device
        )
        timings['global feature extraction'].append(time() - t)
        print (f'Gloabl feature extracting. Done in {time() - t:.4f} sec')
        gc.collect()
        
        # 2. Get image pairs shortlist using VLAD global descriptors
        print("\n--- Step 2: Generating Image Pair Shortlist using VLAD ---")
        # Adjust num_clusters_vlad as needed (e.g., 64, 128, 256)
        # Higher clusters mean higher dimensionality for global descriptor.
        index_pairs = get_image_pairs_shortlist_vlad(
            fnames=images,
            sim_th=0.1,
            min_pairs=10,
            exhaustive_if_less=20,
            feature_dir=feature_dir,
            num_clusters_vlad=128, # Example: 128 clusters for VLAD
            device=device
        )
        print(f"Generated {len(index_pairs)} image pairs using VLAD global descriptor.")
        timings['shortlisting'].append(time() - t)
        print (f'Shortlisting. Number of pairs to match: {len(index_pairs)}. Done in {time() - t:.4f} sec')
        gc.collect()
    
        t = time()

        # --- Step 3.1: Detect Features for each type ---
        print("\n--- Step 3.1: Detecting Features ---")
        kp_h5_paths_by_detector = {}
        desc_h5_paths_by_detector = {}
        aliked_kp_path, aliked_desc_path = detect_aliked(images, feature_dir=feature_dir, device=device)
        kp_h5_paths_by_detector['aliked'] = aliked_kp_path
        desc_h5_paths_by_detector['aliked'] = aliked_desc_path
        gc.collect()
        print("detect aliked done")
        disk_kp_path, disk_desc_path = detect_disk(images, feature_dir=feature_dir, device=device)
        kp_h5_paths_by_detector['disk'] = disk_kp_path
        desc_h5_paths_by_detector['disk'] = disk_desc_path
        print("detect disk done")
        gc.collect()
        # sift_kp_path, sift_desc_path = detect_sift(images, feature_dir=feature_dir, device=device)
        # kp_h5_paths_by_detector['sift'] = sift_kp_path
        # desc_h5_paths_by_detector['sift'] = sift_desc_path
        # print("detect sift done")
        # gc.collect()
        # timings['feature_detection'].append(time() - t)
        # print(f'Features detected in {time() - t:.4f} sec')
        
        # Dedode v2 features would be detected here if you had the code for it.
        # dedode_v2_kp_path, dedode_v2_desc_path = detect_dedode_v2(...)
    
    
        # --- Step 3.2: Perform Matching for each configuration ---
        print("\n--- Step 3.2: Performing Individual Matchings ---")
        match_paths = []
        
        match_h5_paths_by_detector = {}
    
        # 3.2.1. ALIKED + LightGlue (your baseline)
        print("Matching ALIKED + LightGlue...")
        t = time()
        matches_aliked_lg_path = match_with_lightglue_aliked(
            images, index_pairs,
            feature_kp_path=aliked_kp_path, feature_desc_path=aliked_desc_path,
            output_matches_h5='matches_aliked_lightglue.h5',
            device=device
        )
        
        match_h5_paths_by_detector['aliked'] = matches_aliked_lg_path # Use 'aliked' as key for consistency with kp_h5_paths
        gc.collect()
        
        # 3.2.2. DISK + LightGlue
        print("Matching DISK + LightGlue...")
        matches_disk_lg_path = match_with_lightglue_disk(
            images, index_pairs,
            feature_kp_path=disk_kp_path, feature_desc_path=disk_desc_path,
            output_matches_h5='matches_disk_lightglue.h5',
            device=device
        )
        match_h5_paths_by_detector['disk'] = matches_disk_lg_path
        gc.collect()
        
        # # 3.2.3. SIFT + Nearest Neighbor
        # print("Matching SIFT + Nearest Neighbor...")
        # matches_sift_nn_path = match_sift_flann(
        #     images, index_pairs,
        #     feature_kp_path=sift_kp_path, feature_desc_path=sift_desc_path,
        #     output_matches_h5='matches_sift_nn.h5',
        #     device=device # This param is mostly ignored for OpenCV, but for consistency
        # )
        # match_h5_paths_by_detector['sift'] = matches_sift_nn_path
        # gc.collect()
        
        # # 3.2.4. Dedode v2 + Dual Softmax (Placeholder)
        # print("Matching Dedode v2 + Dual Softmax (Placeholder)...")
        # matches_dedode_v2_ds_path = match_dedode_v2_dual_softmax(
        #     img_fnames, index_pairs_to_match,
        #     feature_dir=feature_output_dir, # Assuming Dedode v2 features are saved here
        #     output_matches_h5='matches_dedode_v2_dualsoftmax.h5',
        #     device=device
        # )
        # match_paths.append(matches_dedode_v2_ds_path)
    
    
        # --- Step 3.3: Ensemble the Matching Results ---
        print("\n--- Step 3.3: Ensembling and Remapping all matching results ---")
        unified_kp_path, remapped_matches_path = ensemble_and_remap_matches(
            images,
            match_h5_paths_by_detector=match_h5_paths_by_detector,
            kp_h5_paths_by_detector=kp_h5_paths_by_detector,
            output_unified_kp_h5='keypoints.h5',
            output_remapped_matches_h5='matches.h5',
            kpt_merge_threshold=2.0 # Merge keypoints closer than 2 pixels
        )
        print(f"Final unified keypoints saved to: {unified_kp_path}")
        print(f"Final ensembled and remapped matches saved to: {remapped_matches_path}")

        timings['feature_matching'].append(time() - t)
        print(f'Features matched in {time() - t:.4f} sec')
        gc.collect()
        draw_keypoints_and_matches(images, unified_kp_path, remapped_matches_path, feature_dir)
        
        # --- Step 3.4: Import into COLMAP (using the ensembled matches) ---
        print("\n--- Step 3.4: Importing ensembled results into COLMAP ---")
        database_path = os.path.join(feature_dir, 'colmap.db')
        if os.path.isfile(database_path):
            os.remove(database_path)
        gc.collect()
        sleep(1)
        # import_into_colmap(images_dir, feature_dir=feature_dir, database_path=database_path)
        # output_path = f'{feature_dir}/colmap_rec_aliked'
        # print("import_into_colmap")
        # t = time()
        # pycolmap.match_exhaustive(database_path)
        # timings['RANSAC'].append(time() - t)
        # print(f'Ran RANSAC in {time() - t:.4f} sec')
        
        # # By default colmap does not generate a reconstruction if less than 10 images are registered.
        # # Lower it to 3.
        # mapper_options = pycolmap.IncrementalPipelineOptions()
        # mapper_options.min_model_size = 8
        # mapper_options.max_num_models = 25
        # # mapper_options.mapper.filter_max_reproj_error	 = 9.0

        # os.makedirs(output_path, exist_ok=True)
        # t = time()
        # maps = pycolmap.incremental_mapping(
        #     database_path=database_path, 
        #     image_path=images_dir,
        #     output_path=output_path,
        #     options=mapper_options)
        # sleep(1)
        # timings['Reconstruction'].append(time() - t)
        # print(f'Reconstruction done in  {time() - t:.4f} sec')
        # print(maps)

        # # clear_output(wait=False)
    
        # registered = 0
        # for map_index, cur_map in maps.items():
        #     for index, image in cur_map.images.items():
        #         prediction_index = filename_to_index[image.name]
        #         predictions[prediction_index].cluster_index = map_index
        #         predictions[prediction_index].rotation = deepcopy(image.cam_from_world.rotation.matrix())
        #         predictions[prediction_index].translation = deepcopy(image.cam_from_world.translation)
        #         registered += 1
        # mapping_result_str = f'Dataset "{dataset}" -> Registered {registered} / {len(images)} images with {len(maps)} clusters'
        # mapping_result_strs.append(mapping_result_str)
        # print(mapping_result_str)
        # gc.collect()
    except Exception as e:
        print(e)
        # raise e
        mapping_result_str = f'Dataset "{dataset}" -> Failed!'
        mapping_result_strs.append(mapping_result_str)
        print(mapping_result_str)

print('\nResults')
for s in mapping_result_strs:
    print(s)

print('\nTimings')
for k, v in timings.items():
    print(f'{k} -> total={sum(v):.02f} sec.')

Loading DINOv2 model for patch feature extraction...
DINOv2 model loaded.
Extracting on device cuda:0
Skipping "imc2023_haiper"
Skipping "imc2023_heritage"
Skipping "imc2023_theather_imc2024_church"
Skipping "imc2024_dioscuri_baalshamin"
Skipping "imc2024_lizard_pond"
Skipping "pt_brandenburg_british_buckingham"
Skipping "pt_piazzasanmarco_grandplace"
Skipping "pt_sacrecoeur_trevi_tajmahal"
Skipping "pt_stpeters_stpauls"
Skipping "amy_gardens"
Skipping "fbk_vineyard"

Processing dataset "ETs": 22 images

--- Step 1: Detecting ALIKED and Combining with DINO Patch Features ---


100%|██████████| 22/22 [00:01<00:00, 11.63it/s]


Combined features saved to /kaggle/working/result/featureout/ETs/descriptors_combined.h5
Gloabl feature extracting. Done in 2.1023 sec

--- Step 2: Generating Image Pair Shortlist using VLAD ---


Loading combined local descriptors for K-Means: 100%|██████████| 22/22 [00:00<00:00, 254.47it/s]

Training K-Means with 128 clusters on 42485 descriptors...


...

Saving remapped ensembled matches: 100%|██████████| 97/97 [00:00<00:00, 2729.51it/s]


Ensembled and remapped matches saved to /kaggle/working/result/featureout/ETs/matches.h5
Final unified keypoints saved to: /kaggle/working/result/featureout/ETs/keypoints.h5
Final ensembled and remapped matches saved to: /kaggle/working/result/featureout/ETs/matches.h5
Features matched in 37.7689 sec

--- Loading remapped matches from HDF5 ---


Loading matches: 100%|██████████| 20/20 [00:00<00:00, 1493.78it/s]


--- Drawing Keypoints ---
Keypoints drawn on outliers_out_et001.png, saved to /kaggle/working/result/featureout/ETs/visualization_output/keypoints_outliers_out_et001.png
Keypoints drawn on outliers_out_et003.png, saved to /kaggle/working/result/featureout/ETs/visualization_output/keypoints_outliers_out_et003.png
Keypoints drawn on outliers_out_et002.png, saved to /kaggle/working/result/featureout/ETs/visualization_output/keypoints_outliers_out_et002.png
Keypoints drawn on et_et007.png, saved to /kaggle/working/result/featureout/ETs/visualization_output/keypoints_et_et007.png
Keypoints drawn on et_et003.png, saved to /kaggle/working/result/featureout/ETs/visualization_output/keypoints_et_et003.png
Keypoints drawn on et_et006.png, saved to /kaggle/working/result/featureout/ETs/visualization_output/keypoints_et_et006.png
Keypoints drawn on et_et001.png, saved to /kaggle/working/result/featureout/ETs/visualization_output/keypoints_et_et001.png
Keypoints drawn on et_et004.png, saved to /ka

Keypoints drawn on et_et005.png, saved to /kaggle/working/result/featureout/ETs/visualization_output/keypoints_et_et005.png
Keypoints drawn on et_et000.png, saved to /kaggle/working/result/featureout/ETs/visualization_output/keypoints_et_et000.png
Keypoints drawn on another_et_another_et006.png, saved to /kaggle/working/result/featureout/ETs/visualization_output/keypoints_another_et_another_et006.png
Keypoints drawn on another_et_another_et002.png, saved to /kaggle/working/result/featureout/ETs/visualization_output/keypoints_another_et_another_et002.png
Keypoints drawn on another_et_another_et010.png, saved to /kaggle/working/result/featureout/ETs/visualization_output/keypoints_another_et_another_et010.png
Keypoints drawn on another_et_another_et004.png, saved to /kaggle/working/result/featureout/ETs/visualization_output/keypoints_another_et_another_et004.png
Keypoints drawn on another_et_another_et007.png, saved to /kaggle/working/result/featureout/ETs/visualization_output/keypoints_a

In [31]:
# Must Create a submission file.

array_to_str = lambda array: ';'.join([f"{x:.09f}" for x in array])
none_to_str = lambda n: ';'.join(['nan'] * n)

submission_file = '/kaggle/working/submission.csv'
with open(submission_file, 'w') as f:
    if is_train:
        f.write('dataset,scene,image,rotation_matrix,translation_vector\n')
        for dataset in samples:
            for prediction in samples[dataset]:
                cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'
                rotation = none_to_str(9) if prediction.rotation is None else array_to_str(prediction.rotation.flatten())
                translation = none_to_str(3) if prediction.translation is None else array_to_str(prediction.translation)
                f.write(f'{prediction.dataset},{cluster_name},{prediction.filename},{rotation},{translation}\n')
    else:
        f.write('image_id,dataset,scene,image,rotation_matrix,translation_vector\n')
        for dataset in samples:
            for prediction in samples[dataset]:
                cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'
                rotation = none_to_str(9) if prediction.rotation is None else array_to_str(prediction.rotation.flatten())
                translation = none_to_str(3) if prediction.translation is None else array_to_str(prediction.translation)
                f.write(f'{prediction.image_id},{prediction.dataset},{cluster_name},{prediction.filename},{rotation},{translation}\n')

!head {submission_file}

dataset,scene,image,rotation_matrix,translation_vector
imc2023_haiper,outliers,fountain_image_116.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_108.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_101.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_082.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_071.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_025.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_000.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_007.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_012.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan


In [32]:
# Definitely Compute results if running on the training set.
# Do not do this when submitting a notebook for scoring. All you have to do is save your submission to /kaggle/working/submission.csv.

if is_train:
    t = time()
    final_score, dataset_scores = metric.score(
        gt_csv='/kaggle/input/image-matching-challenge-2025/train_labels.csv',
        user_csv=submission_file,
        thresholds_csv='/kaggle/input/image-matching-challenge-2025/train_thresholds.csv',
        mask_csv=None if is_train else os.path.join(data_dir, 'mask.csv'),
        inl_cf=0,
        strict_cf=-1,
        verbose=True,
    )
    print(f'Computed metric in: {time() - t:.02f} sec.')

imc2023_haiper: score=0.00% (mAA=0.00%, clusterness=0.00%)
imc2023_heritage: score=0.00% (mAA=0.00%, clusterness=0.00%)
imc2023_theather_imc2024_church: score=0.00% (mAA=0.00%, clusterness=0.00%)
imc2024_dioscuri_baalshamin: score=0.00% (mAA=0.00%, clusterness=0.00%)
imc2024_lizard_pond: score=0.00% (mAA=0.00%, clusterness=0.00%)
pt_brandenburg_british_buckingham: score=0.00% (mAA=0.00%, clusterness=0.00%)
pt_piazzasanmarco_grandplace: score=0.00% (mAA=0.00%, clusterness=0.00%)
pt_sacrecoeur_trevi_tajmahal: score=0.00% (mAA=0.00%, clusterness=0.00%)
pt_stpeters_stpauls: score=0.00% (mAA=0.00%, clusterness=0.00%)
amy_gardens: score=0.00% (mAA=0.00%, clusterness=0.00%)
fbk_vineyard: score=0.00% (mAA=0.00%, clusterness=0.00%)
ETs: score=0.00% (mAA=0.00%, clusterness=0.00%)
stairs: score=0.00% (mAA=0.00%, clusterness=0.00%)
Average over all datasets: score=0.00% (mAA=0.00%, clusterness=0.00%)
Computed metric in: 0.06 sec.
